# Demographics Related to College Majors - Nathan Strong Jupyter Test

## What are the some of the demographics and social affects related to majors?

### Fields Being Tracked:

__FOD1P - Recoded field of degree - first entry__<br>
__FOD2P - Recoded field of degree - second entry__<br>
__SEX - Sex__ (1 = male, 2=female)<br>
__AGEP - Age__<br>
__SCH - School enrollment:__<br>
b    .N/A (less than 3 years old)
<br>1    .No, has not attended in the last 3 months
<br>2    .Yes, public school or public college
<br>3    .Yes, private school or college or home school
<br><br>
__RAC1P - Recoded detailed race code__
<br>1    .White alone
<br>2    .Black or African American alone
<br>3    .American Indian alone
<br>4    .Alaska Native alone
<br>5    .American Indian and Alaska Native tribes specified; or American Indian or Al
NATIVITY - Nativity<br>
__NOP - Nativity of parent__
<br>1    .Native
<br>2    .Foreign born

In [ ]:
%matplotlib inline
import pandas as pd
import os
import seaborn as sns

In [ ]:
pums_dir='/Users/strong/Desktop/UCLA Data Science/module 0/Jupyter/csv_pus/'
pums_file='psam_pusa.csv'
df = pd.read_csv(os.path.join(pums_dir, pums_file))

In [ ]:
df.head(5)

In [ ]:
df = df[['FOD1P', 'SEX', 'AGEP', 'SCH', 'RAC1P', 'NATIVITY']]

In [ ]:
df.describe()

In [ ]:
originalCountOfData = len(df)
print(f"original count of data: {originalCountOfData}")

In [ ]:
print('Percentage of null values per-column:')
(df.isna().sum() / len(df)) * 100

In [ ]:
# Remove rows with NaN rows
df = df.dropna()
newCountOfData = len(df)
print(f'Count of data without null values: {newCountOfData}')
print(f'Percecentage of data thrown out: {(originalCountOfData - newCountOfData) / originalCountOfData}')

## Get data for college majors

In [ ]:
majors_dir='/Users/strong/Desktop/UCLA Data Science/module 0/Jupyter'
majors_file='majors-list.csv'
dfMajors = pd.read_csv(os.path.join(majors_dir, majors_file))

In [ ]:
# Convert all strings to numbers and remove the NaN rows
dfMajors['FOD1P'] = pd.to_numeric(dfMajors['FOD1P'], errors='coerce')
dfMajors.dropna()

# Merge Data from Both Tables

In [ ]:
dfFinal = pd.merge(df, dfMajors, how='inner', on='FOD1P')
dfFinal.head(1)

In [ ]:
# Change the Sex Column Cells from 1 & 2 to "Male" & "Female"
dfFinal['SEX'] = dfFinal['SEX'].map({1: 'Male', 2: 'Female'}).to_frame()
# Change the Nativity Column Cells from 1 & 2 to "Native" & "Foreign Born"
dfFinal['NATIVITY'] = dfFinal['NATIVITY'].map({1: 'Native', 2: 'Foreign born'}).to_frame()

# Okay, Lets Start with the Data Analysis Now..
# The Question - What is the Relationship Between Demographics and Majors?

### Lets first see the distrobution of data.
### What is the Male to Female ratio in our data?

In [ ]:
females_in_data_percentage = int(dfFinal['SEX'].value_counts(normalize=True)['Female'].round(2) * 100)
males_in_data_percentage =  int(dfFinal['SEX'].value_counts(normalize=True)['Male'].round(2) * 100)
print(f"Female: {females_in_data_percentage}% / {males_in_data_percentage}% : Male")

### What are their ages?

In [ ]:
print(dfFinal['AGEP'].describe().round(1))

mostCommonAge = dfFinal['AGEP'].value_counts().idxmax()
mostCommonAgeCount = dfFinal['AGEP'].value_counts().max()
mostCommonAgePercentage = (mostCommonAgeCount / dfFinal['AGEP'].count().sum()).round(3) * 100
print(f"Most common age is: {mostCommonAge} ({mostCommonAgeCount} people) at {mostCommonAgePercentage}% of the data.")

## What are the most common majors?

In [ ]:
mostCommonMajor = dfFinal['Major'].value_counts().idxmax()
mostCommonMajorCount = dfFinal['Major'].value_counts().max()
mostCommonMajorPercentage = (mostCommonMajorCount / dfFinal['Major'].count().sum()).round(2) * 100
uniqueMajorCount = dfFinal['Major'].nunique()

print(f"Most common major is: {mostCommonMajor} ({mostCommonMajorCount} people) at {mostCommonMajorPercentage}% of the data.")
print(f"There are {uniqueMajorCount} Majors in total.")

In [ ]:
hist = dfFinal['Major'].hist(bins=uniqueMajorCount, histtype='step')

## Pivot data by Gender vs. Gender

In [ ]:
dfMajorsByGender = dfFinal.groupby(['Major','SEX']).count()
dfMajorsByGenderPercent = dfMajorsByGender / dfFinal.groupby(['Major']).count()
dfMajorsByGenderPercent = dfMajorsByGenderPercent[['AGEP']]
dfMajorsByGenderPercent = dfMajorsByGenderPercent.rename(index=str, columns={"AGEP":"PercentOfTotal"})

In [ ]:
dfMajorsByGenderPercent

In [ ]:
dfFinalMerge = pd.merge(dfMajorsByGender, dfMajorsByGenderPercent, how='inner', on=('Major', 'SEX'))
dfFinalMerge = dfFinalMerge[['AGEP', 'PercentOfTotal']]
dfFinalMerge = dfFinalMerge.rename(index=str, columns={"AGEP":"Total"})

In [ ]:
dfFinalMerge.sort_values('Total', ascending=False)
dfFinalMerge.sort_values(['Total','PercentOfTotal'], ascending=False)
dfFinalMerge.reset_index(inplace=True)
dfFinalMerge

In [ ]:
dfFemale = dfFinalMerge[dfFinalMerge['SEX'] == "Female"]
dfFemale = dfFemale.sort_values(['PercentOfTotal'], ascending=False)
dfFemale.head(20)

In [ ]:
dfMale = dfFinalMerge[dfFinalMerge['SEX'] == "Male"]
dfMale = dfMale.sort_values(['PercentOfTotal'], ascending=False)
dfMale.head(20)